In [1]:
import numpy as np
import pyopencl as cl
import os
import time

os.environ['PYOPENCL_COMPILER_OUTPUT'] = '0'
os.environ['PYOPENCL_CTX'] = '0:1'

class Timer:
    def __enter__(self):
        self.start = time.time()
        return self

    def __exit__(self, *args):
        self.end = time.time()
        self.interval = self.end - self.start
        
def roundUp(numToRound, multiple):
    if (multiple == 0):
        return numToRound
    remainder = numToRound % multiple
    if (remainder == 0):
        return numToRound
    return numToRound + multiple - remainder

BLOCK_SIZE = 16
WPT = 8

In [7]:
kernel = """
#define BLOCK_SIZE 16
#define global_idx(x_idx, y_idx, m) (x_idx * m + y_idx)

#define WPT 8
#define RBLOCK_SIZE (BLOCK_SIZE/WPT)

// GEMM -- 
// A is M x N
// B is N x P
// C (output) is M x P

// -- Uses NDRange Kernel with Local Memory
// M, N, P must all be multiples of BLOCKSIZE
__kernel __attribute__((reqd_work_group_size(BLOCK_SIZE, BLOCK_SIZE, 1)))
void GEMM(
      __global float* restrict A,
      __global float* restrict B,
      __global float* restrict C,
      __const int M,
      __const int N,
      __const int P)
{
    const int row = get_local_id(0);
    const int col = get_local_id(1);
    const int m = BLOCK_SIZE*get_group_id(0) + row;
    const int p = BLOCK_SIZE*get_group_id(1) + col;    
    __local float A_local[BLOCK_SIZE][BLOCK_SIZE];
    __local float B_local[BLOCK_SIZE][BLOCK_SIZE];
    float acc = 0.0f;
    const int numTiles = N/BLOCK_SIZE;
    #pragma unroll
    for (int t=0; t<numTiles; t++) {
        const int r = BLOCK_SIZE*t + row;
        const int c = BLOCK_SIZE*t + col;
        A_local[row][col] = A[m*N + c];
        B_local[row][col] = B[r*P + p];
        barrier(CLK_LOCAL_MEM_FENCE);
        #pragma unroll BLOCK_SIZE
        for (int k=0; k<BLOCK_SIZE; k++){{
            acc += A_local[row][k] * B_local[k][col];
        }}
        barrier(CLK_LOCAL_MEM_FENCE);
    }
    C[m*P + p] = acc;
}

// -- Uses NDRange Kernel with Local Memory and 1D Register tiling
// M, N, P must all be multiples of BLOCKSIZE
__kernel __attribute__((reqd_work_group_size(RBLOCK_SIZE, BLOCK_SIZE, 1)))
void GEMM_1DREG(
      __global float* restrict A,
      __global float* restrict B,
      __global float* restrict C,
      __const int M,
      __const int N,
      __const int P)
{
    const int row = get_local_id(0);
    const int col = get_local_id(1);
    const int m = BLOCK_SIZE*get_group_id(0) + row;
    const int p = BLOCK_SIZE*get_group_id(1) + col;    
    __local float A_local[BLOCK_SIZE][BLOCK_SIZE];
    __local float B_local[BLOCK_SIZE][BLOCK_SIZE];
    
    float acc[WPT];
    for(int w=0; w<WPT; w++){
        acc[w] = 0.0f;
    }
    const int numTiles = N/BLOCK_SIZE;
    #pragma unroll
    for (int t=0; t<numTiles; t++) {
        for (int w=0; w<WPT; w++){
            const int r = BLOCK_SIZE*t + row;
            const int c = BLOCK_SIZE*t + col;
            A_local[row + w*RBLOCK_SIZE][col] = A[(m + w*RBLOCK_SIZE)*N + c];
            B_local[row + w*RBLOCK_SIZE][col] = B[(r + w*RBLOCK_SIZE)*P + p];
        }
        barrier(CLK_LOCAL_MEM_FENCE);
        #pragma unroll BLOCK_SIZE
        for (int k=0; k<BLOCK_SIZE; k++){
            for (int w=0; w<WPT; w++){
                acc[w] += A_local[row + w*RBLOCK_SIZE][k] * B_local[k][col];
            }
        }
        barrier(CLK_LOCAL_MEM_FENCE);
    }
    for (int w=0; w<WPT; w++){
        C[(m + w*RBLOCK_SIZE)*P + p] = acc[w];
    }
}

// -- Uses NDRange Kernel with Local Memory and 2D Register tiling
__kernel __attribute__((reqd_work_group_size(BLOCK_SIZE / WPT, BLOCK_SIZE / WPT, 1)))
void GEMM_2DREG(
      __global float* restrict A,
      __global float* restrict B,
      __global float* restrict C,
      __const int M,
      __const int N,
      __const int P)
{
    const int row = get_local_id(0);
    const int col = get_local_id(1);
    const int m = BLOCK_SIZE*get_group_id(0) + row;
    const int p = BLOCK_SIZE*get_group_id(1) + col;
    __local float A_local[BLOCK_SIZE][BLOCK_SIZE];
    __local float B_local[BLOCK_SIZE][BLOCK_SIZE];
    
    float Areg;
    float Breg[WPT];
    float acc[WPT][WPT];
    for(int wm=0; wm<WPT; wm++){
        for(int wn=0; wn<WPT; wn++){
            acc[wm][wn] = 0.0f;
        }
    }
    const int numTiles = N/BLOCK_SIZE;
    #pragma unroll
    for (int t=0; t<numTiles; t++) {
        for (int wm=0; wm<WPT; wm++){
            for (int wn=0; wn<WPT; wn++){
                const int r = BLOCK_SIZE*t + row;
                const int c = BLOCK_SIZE*t + col;
                A_local[row + wm*RBLOCK_SIZE][col + wn*RBLOCK_SIZE] = A[(m + wm*RBLOCK_SIZE)*N + (c + wn*RBLOCK_SIZE)];
                B_local[row + wm*RBLOCK_SIZE][col + wn*RBLOCK_SIZE] = B[(r + wm*RBLOCK_SIZE)*P + (p + wn*RBLOCK_SIZE)];
            }
        }
        barrier(CLK_LOCAL_MEM_FENCE);
        #pragma unroll BLOCK_SIZE
        for (int k=0; k<BLOCK_SIZE; k++){
            for (int wn=0; wn<WPT; wn++){
                Breg[wn] = B_local[k][col + wn*RBLOCK_SIZE];
            }
            for (int wm=0; wm<WPT; wm++){
                Areg = A_local[row + wm*RBLOCK_SIZE][k];
                for (int wn=0; wn<WPT; wn++){
                    acc[wm][wn] += Areg * Breg[wn];
                }
            }
        }
        barrier(CLK_LOCAL_MEM_FENCE);
    }
    for (int wm=0; wm<WPT; wm++){
        for (int wn=0; wn<WPT; wn++){
            C[(m + wm*RBLOCK_SIZE)*P + (p + wn*RBLOCK_SIZE)] = acc[wm][wn];
        }
    }
}

// -- Uses NDRange Kernel with Local Memory and 2D Register tiling
// M, N, P can be arbitrary sizes
__kernel __attribute__((reqd_work_group_size(BLOCK_SIZE / WPT, BLOCK_SIZE / WPT, 1)))
void GEMM_2DREG_IMITATE_PADDING(
      __global float* restrict A,
      __global float* restrict B,
      __global float* restrict C,
      __const int M,
      __const int N,
      __const int P,
      __const int M_,
      __const int N_,
      __const int P_)
{
    const int row = get_local_id(0);
    const int col = get_local_id(1);
    const int m = BLOCK_SIZE*get_group_id(0) + row;
    const int p = BLOCK_SIZE*get_group_id(1) + col;
    __local float A_local[BLOCK_SIZE][BLOCK_SIZE];
    __local float B_local[BLOCK_SIZE][BLOCK_SIZE];
    
    float Areg;
    float Breg[WPT];
    float acc[WPT][WPT];
    for(int wm=0; wm<WPT; wm++){
        for(int wn=0; wn<WPT; wn++){
            acc[wm][wn] = 0.0f;
        }
    }
    const int numTiles = N_/BLOCK_SIZE;
    #pragma unroll
    for (int t=0; t<numTiles; t++) {
        for (int wm=0; wm<WPT; wm++){
            for (int wn=0; wn<WPT; wn++){
                const int r = BLOCK_SIZE*t + row;
                const int c = BLOCK_SIZE*t + col;
                if(((m + wm*RBLOCK_SIZE) < M) && ((c + wn*RBLOCK_SIZE) < N)){
                    A_local[row + wm*RBLOCK_SIZE][col + wn*RBLOCK_SIZE] = A[(m + wm*RBLOCK_SIZE)*N + (c + wn*RBLOCK_SIZE)];
                } else {
                    A_local[row + wm*RBLOCK_SIZE][col + wn*RBLOCK_SIZE] = 0.0;
                }

                if(((p + wn*RBLOCK_SIZE) < P) && ((r + wm*RBLOCK_SIZE) < N)){
                    B_local[row + wm*RBLOCK_SIZE][col + wn*RBLOCK_SIZE] = B[(r + wm*RBLOCK_SIZE)*P + (p + wn*RBLOCK_SIZE)];
                } else {
                    B_local[row + wm*RBLOCK_SIZE][col + wn*RBLOCK_SIZE] = 0.0;
                }
            }
        }
        barrier(CLK_LOCAL_MEM_FENCE);
        #pragma unroll BLOCK_SIZE
        for (int k=0; k<BLOCK_SIZE; k++){
            for (int wn=0; wn<WPT; wn++){
                Breg[wn] = B_local[k][col + wn*RBLOCK_SIZE];
            }
            for (int wm=0; wm<WPT; wm++){
                Areg = A_local[row + wm*RBLOCK_SIZE][k];
                for (int wn=0; wn<WPT; wn++){
                    acc[wm][wn] += Areg * Breg[wn];
                }
            }
        }
        barrier(CLK_LOCAL_MEM_FENCE);
    }
    for (int wm=0; wm<WPT; wm++){
        for (int wn=0; wn<WPT; wn++){
            if(((m + wm*RBLOCK_SIZE) < M) && ((p + wn*RBLOCK_SIZE) < P)){
                C[(m + wm*RBLOCK_SIZE)*P + (p + wn*RBLOCK_SIZE)] = acc[wm][wn];
            }
            
        }
    }
}

// -- Uses NDRange Kernel with Local Memory
// M, N, P can be arbitrary sizes
__kernel __attribute__((reqd_work_group_size(BLOCK_SIZE, BLOCK_SIZE, 1)))
void GEMM_IMITATE_PADDING(
      __global float* restrict A,
      __global float* restrict B,
      __global float* restrict C,
      __const int M,
      __const int N,
      __const int P,
      __const int M_,
      __const int N_,
      __const int P_)
{
    const int row = get_local_id(0);
    const int col = get_local_id(1);
    const int m = BLOCK_SIZE*get_group_id(0) + row;
    const int p = BLOCK_SIZE*get_group_id(1) + col;
     
    __local float A_local[BLOCK_SIZE][BLOCK_SIZE];
    __local float B_local[BLOCK_SIZE][BLOCK_SIZE];
    float acc = 0.0f;
    const int numTiles = N_/BLOCK_SIZE;
    
    #pragma unroll 4
    
    for (int t=0; t<numTiles; t++) {
        const int r = BLOCK_SIZE*t + row;
        const int c = BLOCK_SIZE*t + col;
        
        if((m < M) && (c < N)){
            int A_idx = m*N + c;
            A_local[row][col] = A[A_idx];
        } else {
            A_local[row][col] = 0.0;
        }
        
        if((p < P) && (r < N)){
            int B_idx = r*P + p;
            B_local[row][col] = B[B_idx];
        } else {
            B_local[row][col] = 0.0;
        }
        
        barrier(CLK_LOCAL_MEM_FENCE);
        #pragma unroll BLOCK_SIZE
        for (int k=0; k<BLOCK_SIZE; k++){{
            acc += A_local[row][k] * B_local[k][col];
        }}
        barrier(CLK_LOCAL_MEM_FENCE);
    }
    
    if((m < M) && (p < P)){
        C[m*P + p] = acc;
    }
}
"""

# MATRIX MULTIPLICATION

In [8]:
mult = 2
M, N, P = 3*(10**mult), 4*(10**mult), 5*(10**mult)
M, N, P = roundUp(M, BLOCK_SIZE), roundUp(N, BLOCK_SIZE), roundUp(P, BLOCK_SIZE)
A = np.random.randn(M, N).astype('float32')
B = np.random.randn(N, P).astype('float32')
ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx)
mf = cl.mem_flags
a_device = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=A)
b_device = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=B)

prg = cl.Program(ctx, kernel).build()
start = time.time()
C = A.dot(B)
print('np: ', time.time() - start)
start = time.time()
C = A @ B
print('py: ', time.time() - start)

C_out = np.zeros_like(C)
c_device = cl.Buffer(ctx, mf.WRITE_ONLY, C_out.nbytes)
C.shape

with Timer() as t:
    event = prg.GEMM(queue, (M, P), (BLOCK_SIZE, BLOCK_SIZE), 
                     a_device, b_device, c_device,
                     np.int32(M), np.int32(N), np.int32(P))
    event.wait()
print('cl: ', t.interval)

cl.enqueue_copy(queue, C_out, c_device)
C_out.shape, M, N, P

np:  0.0015859603881835938
py:  0.0009908676147460938
cl:  0.06376481056213379


((304, 512), 304, 400, 512)

In [9]:
C_out

array([[  9.439566  ,  12.805741  , -10.724502  , ..., -22.892948  ,
          6.8262587 ,  12.482617  ],
       [ -4.716728  ,   3.5809178 ,   8.460974  , ..., -11.874376  ,
        -14.738298  , -19.674625  ],
       [ 10.3869    ,   0.33998686,  -2.2919376 , ...,  -3.548169  ,
         -4.678543  ,   1.2437406 ],
       ...,
       [  5.6830344 ,  14.351683  ,  46.228428  , ...,  -4.9663544 ,
         13.358488  ,   6.82675   ],
       [ 25.55392   ,  -7.1336083 ,  10.141382  , ...,   9.633443  ,
        -25.31231   ,  -2.9148455 ],
       [ 24.146011  ,   6.7507944 ,  18.584644  , ...,  14.319832  ,
          3.0579646 ,  29.500877  ]], dtype=float32)

In [10]:
C

array([[  9.439571  ,  12.805742  , -10.724497  , ..., -22.892923  ,
          6.8262577 ,  12.482628  ],
       [ -4.716733  ,   3.5809164 ,   8.46097   , ..., -11.874374  ,
        -14.738298  , -19.674622  ],
       [ 10.386899  ,   0.33999014,  -2.2919402 , ...,  -3.5481699 ,
         -4.678542  ,   1.2437413 ],
       ...,
       [  5.683032  ,  14.351689  ,  46.228428  , ...,  -4.9663506 ,
         13.358485  ,   6.826749  ],
       [ 25.553919  ,  -7.1336064 ,  10.141384  , ...,   9.63344   ,
        -25.312307  ,  -2.9148445 ],
       [ 24.146008  ,   6.7507973 ,  18.584654  , ...,  14.31982   ,
          3.0579624 ,  29.500868  ]], dtype=float32)

# MATRIX MULTIPLICATION 1D REGISTER TILING

In [11]:
ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx)
mf = cl.mem_flags
a_device = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=A)
b_device = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=B)

prg = cl.Program(ctx, kernel).build()
start = time.time()
C = A.dot(B)
print('np: ', time.time() - start)
start = time.time()
C = A @ B
print('py: ', time.time() - start)

C_out = np.zeros_like(C)
c_device = cl.Buffer(ctx, mf.WRITE_ONLY, C_out.nbytes)
C.shape

with Timer() as t:
    event = prg.GEMM_1DREG(queue, (M // WPT, P), (BLOCK_SIZE // WPT, BLOCK_SIZE), 
                     a_device, b_device, c_device,
                     np.int32(M), np.int32(N), np.int32(P))
    event.wait()
print('cl: ', t.interval)

cl.enqueue_copy(queue, C_out, c_device)
C_out.shape, M, N, P

np:  0.0018579959869384766
py:  0.0016303062438964844
cl:  0.08417892456054688


((304, 512), 304, 400, 512)

In [12]:
C

array([[  9.439571  ,  12.805742  , -10.724497  , ..., -22.892923  ,
          6.8262577 ,  12.482628  ],
       [ -4.716733  ,   3.5809164 ,   8.46097   , ..., -11.874374  ,
        -14.738298  , -19.674622  ],
       [ 10.386899  ,   0.33999014,  -2.2919402 , ...,  -3.5481699 ,
         -4.678542  ,   1.2437413 ],
       ...,
       [  5.683032  ,  14.351689  ,  46.228428  , ...,  -4.9663506 ,
         13.358485  ,   6.826749  ],
       [ 25.553919  ,  -7.1336064 ,  10.141384  , ...,   9.63344   ,
        -25.312307  ,  -2.9148445 ],
       [ 24.146008  ,   6.7507973 ,  18.584654  , ...,  14.31982   ,
          3.0579624 ,  29.500868  ]], dtype=float32)

In [13]:
C_out

array([[  9.439566  ,  12.805741  , -10.724502  , ..., -22.892948  ,
          6.8262587 ,  12.482617  ],
       [ -4.716728  ,   3.5809178 ,   8.460974  , ..., -11.874376  ,
        -14.738298  , -19.674625  ],
       [ 10.3869    ,   0.33998686,  -2.2919376 , ...,  -3.548169  ,
         -4.678543  ,   1.2437406 ],
       ...,
       [  5.6830344 ,  14.351683  ,  46.228428  , ...,  -4.9663544 ,
         13.358488  ,   6.82675   ],
       [ 25.55392   ,  -7.1336083 ,  10.141382  , ...,   9.633443  ,
        -25.31231   ,  -2.9148455 ],
       [ 24.146011  ,   6.7507944 ,  18.584644  , ...,  14.319832  ,
          3.0579646 ,  29.500877  ]], dtype=float32)

# MATRIX MULTIPLICATION REGISTER TILING

In [14]:
ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx)
mf = cl.mem_flags
a_device = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=A)
b_device = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=B)

prg = cl.Program(ctx, kernel).build()
start = time.time()
C = A.dot(B)
print('np: ', time.time() - start)
start = time.time()
C = A @ B
print('py: ', time.time() - start)

C_out = np.zeros_like(C)
c_device = cl.Buffer(ctx, mf.WRITE_ONLY, C_out.nbytes)
C.shape

with Timer() as t:
    event = prg.GEMM_2DREG(queue, (M // WPT, P // WPT), (BLOCK_SIZE // WPT, BLOCK_SIZE // WPT), 
                     a_device, b_device, c_device,
                     np.int32(M), np.int32(N), np.int32(P))
    event.wait()
print('cl: ', t.interval)

cl.enqueue_copy(queue, C_out, c_device)
C_out.shape, M, N, P

np:  0.0019860267639160156
py:  0.0014548301696777344
cl:  0.13943099975585938


((304, 512), 304, 400, 512)

In [15]:
C

array([[  9.439571  ,  12.805742  , -10.724497  , ..., -22.892923  ,
          6.8262577 ,  12.482628  ],
       [ -4.716733  ,   3.5809164 ,   8.46097   , ..., -11.874374  ,
        -14.738298  , -19.674622  ],
       [ 10.386899  ,   0.33999014,  -2.2919402 , ...,  -3.5481699 ,
         -4.678542  ,   1.2437413 ],
       ...,
       [  5.683032  ,  14.351689  ,  46.228428  , ...,  -4.9663506 ,
         13.358485  ,   6.826749  ],
       [ 25.553919  ,  -7.1336064 ,  10.141384  , ...,   9.63344   ,
        -25.312307  ,  -2.9148445 ],
       [ 24.146008  ,   6.7507973 ,  18.584654  , ...,  14.31982   ,
          3.0579624 ,  29.500868  ]], dtype=float32)

In [16]:
C_out

array([[  9.439566  ,  12.805741  , -10.724502  , ..., -22.892948  ,
          6.8262587 ,  12.482617  ],
       [ -4.716728  ,   3.5809178 ,   8.460974  , ..., -11.874376  ,
        -14.738298  , -19.674625  ],
       [ 10.3869    ,   0.33998686,  -2.2919376 , ...,  -3.548169  ,
         -4.678543  ,   1.2437406 ],
       ...,
       [  5.6830344 ,  14.351683  ,  46.228428  , ...,  -4.9663544 ,
         13.358488  ,   6.82675   ],
       [ 25.55392   ,  -7.1336083 ,  10.141382  , ...,   9.633443  ,
        -25.31231   ,  -2.9148455 ],
       [ 24.146011  ,   6.7507944 ,  18.584644  , ...,  14.319832  ,
          3.0579646 ,  29.500877  ]], dtype=float32)

# MATRIX MULTIPLICATION PADDED

In [17]:
np_times, py_times, cl_times = [], [], []

mult = 2
M, N, P = 3*(10**mult), 4*(10**mult), 5*(10**mult)
M_, N_, P_ = roundUp(M, BLOCK_SIZE), roundUp(N, BLOCK_SIZE), roundUp(P, BLOCK_SIZE)
A = np.random.randn(M, N).astype('float32')
B = np.random.randn(N, P).astype('float32')
ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx)
mf = cl.mem_flags
a_device = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=A)
b_device = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=B)

prg = cl.Program(ctx, kernel).build()
start = time.time()
C = A.dot(B)
end = time.time()
print('np: ', end - start)
np_times.append(end - start)
start = time.time()
C = A @ B
end = time.time()
print('py: ', end - start)
py_times.append(end - start)

C_out = np.zeros_like(C)
c_device = cl.Buffer(ctx, mf.WRITE_ONLY, C_out.nbytes)
C.shape

with Timer() as t:
    event = prg.GEMM_IMITATE_PADDING(queue, (M_ , P_), (BLOCK_SIZE, BLOCK_SIZE), 
                     a_device, b_device, c_device,
                     np.int32(M), np.int32(N), np.int32(P),
                     np.int32(M_), np.int32(N_), np.int32(P_))
    event.wait()
print('cl: ', t.interval)
cl_times.append(t.interval)
cl.enqueue_copy(queue, C_out, c_device)

C_out.shape, M, N, P

np:  0.0012350082397460938
py:  0.0012259483337402344
cl:  0.056999921798706055


((300, 500), 300, 400, 500)

In [18]:
C_out

array([[ 10.297104 ,  -1.027521 ,   6.006107 , ..., -19.869083 ,
         -1.552961 ,  18.661287 ],
       [  1.3731815,  -5.17045  , -16.228472 , ...,  -8.745936 ,
         25.594524 ,  15.454743 ],
       [ 14.936704 ,  27.659956 ,   0.9331303, ...,  47.217636 ,
        -18.784971 ,  -9.16756  ],
       ...,
       [ -3.0717552,  14.189445 ,  20.689833 , ...,  18.131601 ,
         -5.846238 ,  18.857466 ],
       [ -1.8767414,  27.955114 ,  18.338612 , ...,  20.37863  ,
          4.8617253, -29.914093 ],
       [-11.818918 ,  14.612467 ,   9.593132 , ...,   5.0218234,
         14.560721 , -11.932028 ]], dtype=float32)

In [19]:
C

array([[ 10.297104  ,  -1.0275171 ,   6.0061083 , ..., -19.86907   ,
         -1.5529585 ,  18.661293  ],
       [  1.373178  ,  -5.1704493 , -16.228472  , ...,  -8.745934  ,
         25.59454   ,  15.454743  ],
       [ 14.936706  ,  27.659946  ,   0.93313074, ...,  47.217636  ,
        -18.78497   ,  -9.167558  ],
       ...,
       [ -3.0717509 ,  14.189448  ,  20.689816  , ...,  18.131584  ,
         -5.8462415 ,  18.857464  ],
       [ -1.8767414 ,  27.955116  ,  18.33861   , ...,  20.37863   ,
          4.861725  , -29.914097  ],
       [-11.81892   ,  14.612468  ,   9.593132  , ...,   5.0218186 ,
         14.560723  , -11.932032  ]], dtype=float32)

# MATRIX MULTIPLICATION PADDED REGISTER TILING

In [23]:
np_times, py_times, cl_times = [], [], []

ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx)
mf = cl.mem_flags
a_device = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=A)
b_device = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=B)

prg = cl.Program(ctx, kernel).build()
start = time.time()
C = A.dot(B)
end = time.time()
print('np: ', end - start)
np_times.append(end - start)
start = time.time()
C = A @ B
end = time.time()
print('py: ', end - start)
py_times.append(end - start)

C_out = np.zeros_like(C)
c_device = cl.Buffer(ctx, mf.WRITE_ONLY, C_out.nbytes)
C.shape

with Timer() as t:
    event = prg.GEMM_2DREG_IMITATE_PADDING(queue, (M_ // WPT , P_ // WPT), (BLOCK_SIZE // WPT, BLOCK_SIZE // WPT), 
                     a_device, b_device, c_device,
                     np.int32(M), np.int32(N), np.int32(P),
                     np.int32(M_), np.int32(N_), np.int32(P_))
    event.wait()
print('cl: ', t.interval)
cl_times.append(t.interval)
cl.enqueue_copy(queue, C_out, c_device)

C_out.shape, M, N, P

np:  0.0016679763793945312
py:  0.0016739368438720703
cl:  0.1386098861694336


((300, 500), 300, 400, 500)

In [24]:
C

array([[ 10.297104  ,  -1.0275171 ,   6.0061083 , ..., -19.86907   ,
         -1.5529585 ,  18.661293  ],
       [  1.373178  ,  -5.1704493 , -16.228472  , ...,  -8.745934  ,
         25.59454   ,  15.454743  ],
       [ 14.936706  ,  27.659946  ,   0.93313074, ...,  47.217636  ,
        -18.78497   ,  -9.167558  ],
       ...,
       [ -3.0717509 ,  14.189448  ,  20.689816  , ...,  18.131584  ,
         -5.8462415 ,  18.857464  ],
       [ -1.8767414 ,  27.955116  ,  18.33861   , ...,  20.37863   ,
          4.861725  , -29.914097  ],
       [-11.81892   ,  14.612468  ,   9.593132  , ...,   5.0218186 ,
         14.560723  , -11.932032  ]], dtype=float32)

In [25]:
C_out

array([[ 10.297104 ,  -1.027521 ,   6.006107 , ..., -19.869083 ,
         -1.552961 ,  18.661287 ],
       [  1.3731815,  -5.17045  , -16.228472 , ...,  -8.745936 ,
         25.594524 ,  15.454743 ],
       [ 14.936704 ,  27.659956 ,   0.9331303, ...,  47.217636 ,
        -18.784971 ,  -9.16756  ],
       ...,
       [ -3.0717552,  14.189445 ,  20.689833 , ...,  18.131601 ,
         -5.846238 ,  18.857466 ],
       [ -1.8767414,  27.955114 ,  18.338612 , ...,  20.37863  ,
          4.8617253, -29.914093 ],
       [-11.818918 ,  14.612467 ,   9.593132 , ...,   5.0218234,
         14.560721 , -11.932028 ]], dtype=float32)